# Validation autoencoder model
Test and experiments to check the performance of an autoencoder

In [ ]:
import junodch_utils_read_img as utils

import matplotlib.pyplot as matPlt
import numpy as np
import pandas as pd
import tensorflow as tf

import keras
from keras import layers, losses
from keras.models import Model
from shapely.geometry import Polygon, box

from tqdm.keras import TqdmCallback
from sklearn.metrics import confusion_matrix

import rasterio
from rasterio import plot as rastPlt
from rasterio.plot import reshape_as_raster
from rasterio.merge import merge as rasterMerge
from rasterio.mask import mask as rasterMask

In [ ]:
autoencoder = keras.models.load_model('model/autoencoder_64px_encoder_1024')

In [ ]:
folderName = "img/sudan/"
pathSatellite = folderName + "Sentinel-2.tif"
pathNight = folderName + "Night_VIIRS.tif"
pathValidation = folderName + "Population_GHSL.tif"

aoi = utils.getImgBorder(pathSatellite)

# Fetch coords
dataCoords, dataRadiance = utils.getTilesCoordsPerimeter(pathNight, area=aoi)
print(len(dataCoords))

#### Validation

In [ ]:
trainMask = dataRadiance>0.2
lightCoords = dataCoords[trainMask]

print('Tiles:',dataCoords.shape[0])
print('Light Tile:',lightCoords.shape[0])

with rasterio.open(pathSatellite) as f:
  trainData, _ = utils.coordsToImgsFormated(f, lightCoords, res=64)
print(trainData.shape)

### Display some Tiles
Selection with indexesTest what tile to display. showDetail will show the encoding steps but is best set when with one sample.  
showDetail = 0 : Show original and final result
showDetail = 1 : Show original, the encoded data en the final result
showDetail = 2 : Show all steps.

In [ ]:
print("Test display some tiles")

indexesTest = [*np.argwhere(trainMask)[100:600:25].flatten(), *range(0,100000,5000)]

with rasterio.open(pathSatellite) as f:
  dataTest, _ = utils.coordsToImgsFormated(f, dataCoords[indexesTest], res=64)

utils.displayAutoEncoderResults(autoencoder, dataTest, showDetail=0, precision=5)